In [1]:
from factuality_evaluator_rs import UnilateralFactualityEvaluator, BilateralFactualityEvaluator
from tqdm import tqdm
import json, os, pandas as pd

In [2]:
df = pd.read_csv(
    "https://openaipublic.blob.core.windows.net/simple-evals/simple_qa_test_set.csv"
)

In [3]:
SIMPLE_QA_DATASET = df.to_dict(orient="records")[:1000]
DATASET_SIZE = len(SIMPLE_QA_DATASET)
EXPERIMENTAL_RUN_VERSION = "v7"

In [4]:
MODELS = [ 
    { "model_name": "gpt-4o-mini", "batch_size": 100 },
    # { "model_name": "gpt-4o-2024-11-20", "batch_size": 100 },
    # { "model_name": "mistralai/Mistral-7B-Instruct-v0.3", "batch_size": 50 },
    # { "model_name": "mistralai/Mixtral-8x7B-Instruct-v0.1", "batch_size": 50 },
    # { "model_name": "claude-3-5-haiku-20241022", "batch_size": 1 },
    # { "model_name": "claude-3-5-sonnet-20241022", "batch_size": 1 },
    # { "model_name": "meta-llama/Llama-3.3-70B-Instruct", "batch_size": 50 },
]

In [5]:
# Check if directory exists and create if not
if not os.path.exists(f'experiments/{EXPERIMENTAL_RUN_VERSION}'):
    os.makedirs(f'experiments/{EXPERIMENTAL_RUN_VERSION}/unilateral')
    os.makedirs(f'experiments/{EXPERIMENTAL_RUN_VERSION}/bilateral')

In [6]:
for model in MODELS:
    filename = f'experiments/{EXPERIMENTAL_RUN_VERSION}/unilateral/{model["model_name"].split("/")[-1]}-simpleqa.json'
    if os.path.isfile(filename):
        print(f'{model["model_name"]:36}: EXISTS')
    else:
        evaluator = UnilateralFactualityEvaluator(model["model_name"], batch_size=model["batch_size"])
        results = []
        for datapoint in tqdm(SIMPLE_QA_DATASET, desc=f'{evaluator.model_name:36}', total=len(SIMPLE_QA_DATASET)):
            results.append(evaluator.invoke(datapoint, samples=5))
        json.dump(results, open(filename, "w+"))

gpt-4o-mini                         : 100%|██████████| 1000/1000 [7:42:49<00:00, 27.77s/it] 


In [ ]:
for model in MODELS:
    filename = f'experiments/{EXPERIMENTAL_RUN_VERSION}/bilateral/{model["model_name"].split("/")[-1]}-simpleqa.json'
    if os.path.isfile(filename):
        print(f'{model["model_name"]:36}: EXISTS')
    else:
        evaluator = BilateralFactualityEvaluator(model["model_name"], batch_size=model["batch_size"])
        results = []
        for datapoint in tqdm(SIMPLE_QA_DATASET, desc=f'{evaluator.model_name:36}', total=len(SIMPLE_QA_DATASET)):
            results.append(evaluator.invoke(datapoint, samples=5))
        json.dump(results, open(filename, "w+"))

gpt-4o-mini                         :  56%|█████▌    | 560/1000 [10:48:20<7:13:40, 59.14s/it] 